In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from pathlib import Path

# MapBox token
token = "pk.eyJ1IjoiZnlubndpIiwiYSI6ImNrODk3YmF6MzAzcDczbWs5NXdhaGpyNzYifQ.vHweJb-1hjDeE21tTs7tGQ"

## Load data

In [2]:
DIR_base = Path.cwd().parent
DIR_data = Path.joinpath(DIR_base, "data")

In [3]:
data = pd.read_pickle(Path.joinpath(DIR_data, "processed", "temp_yearly_summary.pkl"))
stations = pd.read_pickle(Path.joinpath(DIR_data, "processed", "stations_hourly_temp_hist.pkl"))

## Extract data

In [4]:
lats = stations['latitude']
lons = stations['longitude']
ids = stations['station_id']
names = stations['name']
states = stations['state']
altitudes = stations['altitude']

## Assemble Plot

In [6]:
# figure
fig = go.Figure()

# marker
markers = go.scattermapbox.Marker(
    size=15,
    color=altitudes,
    colorscale='thermal',
    opacity=0.8
)

# trace
trace = go.Scattermapbox(
    lat=lats,
    lon=lons,
    mode='markers',
    marker=markers,
    text=altitudes
)

# mapbox
my_mapbox = {
    'accesstoken': token,
    'style': 'basic',
    'zoom': 5,
    'center': {
        'lon': 10.02,
        'lat': 51.13
    }
}


# layout
fig.update_layout(
    title="DWD Stations in Germany",
    mapbox=my_mapbox,
    autosize=True
)

fig.write_html("lel.html")